In [1]:
import zipfile
import os
from collections import namedtuple

import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
from sklearn.cluster import KMeans

from JicBitstream import JicBitstream, JicBitstreamZip

%matplotlib notebook

In [2]:
def compare(lst, fn0, fn1, path="../../results/out/", verbose=False):
    if verbose:
        print(f"comparing {os.path.basename(fn0)} and {os.path.basename(fn1)}")
    fn0 = os.path.join(path, fn0)
    fn1 = os.path.join(path, fn1)
    jic0 = JicBitstreamZip(fn0)
    jic1 = JicBitstreamZip(fn1)
    diff = jic0.diff_pos(jic1)
    if verbose:
        print(diff)
    lst.append(diff)

In [3]:
pin_list = open("../../resources/pin_list_5SGSMD5K1F40C1_8A.txt", "r").readlines()
pin_list = [pin.strip() for pin in pin_list]

In [4]:
PU_ADDR = {}
CHKSUM_ADDR = {}

In [5]:
for pin in pin_list:
    print("\n", pin)
    diff_lst = []

    # PU vs no PU
    compare(diff_lst, f"{pin}_2V5_16mA_pu_off_dly_no.zip", f"{pin}_2V5_16mA_pu_on_dly_no.zip")
    compare(diff_lst, f"{pin}_2V5_12mA_pu_off_dly_no.zip", f"{pin}_2V5_12mA_pu_on_dly_no.zip")
    compare(diff_lst, f"{pin}_2V5_8mA_pu_off_dly_no.zip", f"{pin}_2V5_8mA_pu_on_dly_no.zip")
    compare(diff_lst, f"{pin}_2V5_4mA_pu_off_dly_no.zip", f"{pin}_2V5_4mA_pu_on_dly_no.zip")

    diff_lst_comb = []

    for diff in diff_lst:
        for el in diff[0]:
            diff_lst_comb.append(el)

    diff_lst_comb = np.array(sorted(list(set(diff_lst_comb))))
    X = diff_lst_comb.reshape(-1, 1)

    kmeans = KMeans(n_clusters=3, random_state=123).fit(X)
    print("K-means cluster centers", kmeans.cluster_centers_.reshape(-1))

    APPROX_PIN_LOC = int(1e6)
    PIN_GROUP = kmeans.predict(np.array([APPROX_PIN_LOC]).reshape(1, -1))
    feat_addrs = X[kmeans.labels_ == PIN_GROUP].reshape((-1))

    jic_path="../../results/out/"

    jics = [
        (f"{pin}_2V5_16mA_pu_off_dly_no.zip", 0),
        (f"{pin}_2V5_16mA_pu_on_dly_no.zip", 1),
        (f"{pin}_2V5_12mA_pu_off_dly_no.zip", 0),
        (f"{pin}_2V5_12mA_pu_on_dly_no.zip", 1),
        (f"{pin}_2V5_8mA_pu_off_dly_no.zip", 0),
        (f"{pin}_2V5_8mA_pu_on_dly_no.zip", 1),
        (f"{pin}_2V5_4mA_pu_off_dly_no.zip", 0),
        (f"{pin}_2V5_4mA_pu_on_dly_no.zip", 1),
    ]
    
    print(feat_addrs)
    diff = np.diff(feat_addrs)

    for jic, exp in jics:
        j = JicBitstreamZip(os.path.join(jic_path, jic))
        feat = j.get_els(feat_addrs).astype(int)
        print(f"{jic.replace('.zip', ''):28}", end=" ")
        print(feat)
        assert feat[0] == exp
        
    # some heuristics - first el is PU addr, the rest should be checksum
    assert diff[0] > 16, "first addr is far away from the rest (checksum)"    
    np.testing.assert_array_less(diff[1:], 16)
        
    PU_ADDR[pin] = feat_addrs[0]
    CHKSUM_ADDR[pin] = feat_addrs[1:]



 R32
K-means cluster centers [3.65000000e+03 2.68437282e+08 1.16534600e+06]
[1162477 1165345 1165346 1165347 1165350 1165352 1165353 1165354 1165355
 1165357]
R32_2V5_16mA_pu_off_dly_no   [0 1 0 1 0 1 0 0 1 1]
R32_2V5_16mA_pu_on_dly_no    [1 0 1 0 1 0 1 1 0 0]
R32_2V5_12mA_pu_off_dly_no   [0 1 0 0 1 1 1 1 1 1]
R32_2V5_12mA_pu_on_dly_no    [1 0 1 1 0 0 0 0 0 0]
R32_2V5_8mA_pu_off_dly_no    [0 0 0 1 1 0 0 0 1 1]
R32_2V5_8mA_pu_on_dly_no     [1 1 1 0 0 1 1 1 0 0]
R32_2V5_4mA_pu_off_dly_no    [0 1 1 0 0 1 1 1 0 1]
R32_2V5_4mA_pu_on_dly_no     [1 0 0 1 1 0 0 0 1 0]

 P32
K-means cluster centers [2.68437275e+08 3.65700000e+03 1.17484800e+06]
[1166573 1174848 1174849 1174850 1174851 1174852 1174854 1174855 1174857
 1174860]
P32_2V5_16mA_pu_off_dly_no   [0 1 1 0 1 0 0 1 0 1]
P32_2V5_16mA_pu_on_dly_no    [1 0 0 1 0 1 1 0 1 0]
P32_2V5_12mA_pu_off_dly_no   [0 1 1 1 1 1 1 0 0 1]
P32_2V5_12mA_pu_on_dly_no    [1 0 0 0 0 0 0 1 1 0]
P32_2V5_8mA_pu_off_dly_no    [0 1 1 1 1 1 1 1 0 1]
P32_2V5_8mA_pu_on

K-means cluster centers [3.65100000e+03 2.68437283e+08 1.03229100e+06]
[1032109 1032290 1032291 1032294 1032296 1032297 1032298 1032299 1032302
 1032303]
A32_2V5_16mA_pu_off_dly_no   [0 0 0 0 0 1 1 0 0 0]
A32_2V5_16mA_pu_on_dly_no    [1 1 1 1 1 0 0 1 1 1]
A32_2V5_12mA_pu_off_dly_no   [0 0 0 0 0 1 0 1 1 1]
A32_2V5_12mA_pu_on_dly_no    [1 1 1 1 1 0 1 0 0 0]
A32_2V5_8mA_pu_off_dly_no    [0 0 1 0 1 0 0 1 0 1]
A32_2V5_8mA_pu_on_dly_no     [1 1 0 1 0 1 1 0 1 0]
A32_2V5_4mA_pu_off_dly_no    [0 1 1 1 1 1 1 0 0 0]
A32_2V5_4mA_pu_on_dly_no     [1 0 0 0 0 0 0 1 1 1]

 E34
K-means cluster centers [2.68437281e+08 3.65400000e+03 9.84769000e+05]
[977645 984769 984774 984775 984778 984779 984780 984782]
E34_2V5_16mA_pu_off_dly_no   [0 0 0 0 1 1 0 1]
E34_2V5_16mA_pu_on_dly_no    [1 1 1 1 0 0 1 0]
E34_2V5_12mA_pu_off_dly_no   [0 1 0 0 0 0 1 0]
E34_2V5_12mA_pu_on_dly_no    [1 0 1 1 1 1 0 1]
E34_2V5_8mA_pu_off_dly_no    [0 0 0 1 1 1 1 0]
E34_2V5_8mA_pu_on_dly_no     [1 1 1 0 0 0 0 1]
E34_2V5_4mA_pu_off_dl

K33_2V5_12mA_pu_off_dly_no   [0 0 0 1 1 0 0 0 1 0]
K33_2V5_12mA_pu_on_dly_no    [1 1 1 0 0 1 1 1 0 1]
K33_2V5_8mA_pu_off_dly_no    [0 0 0 1 1 1 1 1 1 1]
K33_2V5_8mA_pu_on_dly_no     [1 1 1 0 0 0 0 0 0 0]
K33_2V5_4mA_pu_off_dly_no    [0 0 1 0 1 1 1 0 0 1]
K33_2V5_4mA_pu_on_dly_no     [1 1 0 1 0 0 0 1 1 0]

 J33
K-means cluster centers [3.64900000e+03 2.68437278e+08 8.51712000e+05]
[845517 851712 851714 851715 851716 851717 851719 851721 851722 851726]
J33_2V5_16mA_pu_off_dly_no   [0 1 1 1 1 1 0 1 0 0]
J33_2V5_16mA_pu_on_dly_no    [1 0 0 0 0 0 1 0 1 1]
J33_2V5_12mA_pu_off_dly_no   [0 1 0 1 1 0 0 1 1 0]
J33_2V5_12mA_pu_on_dly_no    [1 0 1 0 0 1 1 0 0 1]
J33_2V5_8mA_pu_off_dly_no    [0 0 0 1 1 1 0 0 1 0]
J33_2V5_8mA_pu_on_dly_no     [1 1 1 0 0 0 1 1 0 1]
J33_2V5_4mA_pu_off_dly_no    [0 1 0 1 1 1 1 1 0 0]
J33_2V5_4mA_pu_on_dly_no     [1 0 1 0 0 0 0 0 1 1]

 H34
K-means cluster centers [3.64900000e+03 2.68437276e+08 8.13696000e+05]
[813325 813696 813698 813699 813702 813706 813707 813711]
H3

In [7]:
print(PU_ADDR)
print(CHKSUM_ADDR)

{'R32': 1162477, 'P32': 1166573, 'U31': 1163821, 'T31': 1165165, 'N32': 1110701, 'M32': 1114733, 'N33': 1112045, 'M33': 1113389, 'L33': 1081261, 'L34': 1085293, 'P34': 1082605, 'N34': 1083949, 'D33': 1029421, 'C33': 1033517, 'B32': 1030765, 'A32': 1032109, 'E34': 977645, 'D34': 981677, 'A34': 978989, 'A35': 980333, 'C34': 948205, 'B34': 952237, 'A36': 949549, 'A37': 950893, 'E32': 894605, 'F32': 898637, 'G33': 895949, 'G32': 897293, 'F33': 842829, 'E33': 846861, 'K33': 844173, 'J33': 845517, 'H34': 813325, 'G34': 817421, 'K34': 814733, 'J34': 816077}
{'R32': array([1165345, 1165346, 1165347, 1165350, 1165352, 1165353, 1165354,
       1165355, 1165357]), 'P32': array([1174848, 1174849, 1174850, 1174851, 1174852, 1174854, 1174855,
       1174857, 1174860]), 'U31': array([1165344, 1165345, 1165347, 1165350, 1165351, 1165354, 1165355]), 'T31': array([1165346, 1165347, 1165350, 1165352, 1165353, 1165354, 1165355,
       1165358, 1165359]), 'N32': array([1117825, 1117830, 1117831, 1117834, 1